In [5]:
from Bio import SeqIO
import statistics
import pandas as pd
import numpy as np
import math
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
import os
from collections import defaultdict
import random

In [ ]:
#graph = obonet.read_obo("/content/drive/MyDrive/cafa-5-protein-function-prediction/Train/go-basic.obo")

In [20]:
os.chdir("/Users/zhongyuanchen")
fasta_sequences = SeqIO.parse(open("Train/train_sequences.fasta"),'fasta')
df = pd.DataFrame((fasta.id,str(fasta.seq)) for fasta in fasta_sequences)
df.columns = ["EntryID","Sequence"]
df.set_index("EntryID",inplace=True)
df.head()

,Sequence
EntryID,
P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...


In [21]:
# Trim or pad each sequence to a specific size TRIM_LENGTH
TRIM_LENGTH = 400
df["Sequence"] = df["Sequence"].apply(lambda x:x[:TRIM_LENGTH] if len(x)>TRIM_LENGTH else x.ljust(TRIM_LENGTH,"0"))

In [22]:
# Define a function to one hot encode a given fasta sequence, the resulting array will have size (TRIM_LENGTH,27)
def sq_one_hot(sequence):
    mapping = sq_one_hot.mapping
    seq = [mapping[char] for char in sequence]
    return np.eye(len(mapping))[seq].reshape(TRIM_LENGTH,27,1)

sq_one_hot.mapping = {chr(i + ord("A")):i for i in range(26)}
sq_one_hot.mapping["0"] = 26
print(sq_one_hot.mapping)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, '0': 26}


In [23]:
# Read in the GO-terms data
terms_df = pd.read_csv('Train/train_terms.tsv',sep="\t")
terms_df.drop(columns = ["aspect"],inplace = True)
terms_df.set_index("EntryID",inplace = True)
freq_counts = terms_df["term"].value_counts()

In [24]:
# Select num_labels most frequent GO terms and restrict to these prediction
num_labels = 1500
chosen_terms = freq_counts.index[:num_labels]
chosent_terms = set(chosen_terms)
filt = terms_df["term"].isin(chosen_terms)
terms_df = terms_df[filt]
sum(freq_counts.iloc[:num_labels])/sum(freq_counts.iloc)

0.824170378699083

In [25]:
# Multilabel encoding
terms_mlb = MultiLabelBinarizer()
terms_mlb.fit([terms_df['term']])

MultiLabelBinarizer()

In [26]:
# Define dictionaries for fast acessing of terms and sequences using their EntryIDs
id_sq = {entry_id:df.loc[entry_id]["Sequence"] for entry_id in df.index}
id_terms = defaultdict(list)
for entry_id,row in terms_df.iterrows():
    id_terms[entry_id].append(row["term"])


# Define a data generator object to generate data for each epoch
# Without generator, the size of all sequences add up to 30 gigabytes
class Data_Generator(tf.keras.utils.Sequence):

    def __init__(self,indexs, batch_size):
        self.indexs = indexs
        self.batch_size = batch_size

    def __len__(self):
        return math.floor(len(self.indexs) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.indexs))
        batch_x = []
        batch_y = []

        for i in range(low,high):
            entry_id = self.indexs[i]
            anotation = id_terms[entry_id]
            batch_x.append(sq_one_hot(id_sq[entry_id]))
            batch_y.append(terms_mlb.transform([anotation])[0])

        return np.array(batch_x).reshape(self.batch_size,TRIM_LENGTH,27,1), np.array(batch_y)


In [43]:
# Create training and testing datas by shuffling and spliting an array of EntryIDs
indexs = list(df.index)
random.shuffle(indexs)
train_test_split = int(0.3*len(indexs))
train_indexs = indexs[:train_test_split]
test_indexs = indexs[train_test_split:]

# Define the respected generators for training and testing
train_generator = Data_Generator(train_indexs,512)
test_generator = Data_Generator(test_indexs,512)

# Create a simple CNN for multilabel classification
model = models.Sequential()
model.add(layers.Conv2D(32, (8, 27), activation='relu', input_shape=(TRIM_LENGTH, 27, 1)))
#model.add(layers.MaxPooling2D(pool_size=(2,1), strides=None, padding="valid"))
#model.add(layers.Conv2D(32,(32,1),activation='relu'))
#model.add(layers.MaxPooling2D(pool_size=(6,1), strides=None, padding="valid"))
model.add(layers.Flatten())
model.add(layers.Dense(num_labels,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['binary_accuracy', tf.keras.metrics.AUC()])
model.fit(x = train_generator,validation_data = test_generator,epochs = 10)

Epoch 1/10
83/83 [==============================] - 142s 2s/step - loss: 0.1070 - binary_accuracy: 0.9742 - auc_7: 0.7736 - val_loss: 0.0774 - val_binary_accuracy: 0.9800 - val_auc_7: 0.8380
Epoch 2/10
83/83 [==============================] - 130s 2s/step - loss: 0.0769 - binary_accuracy: 0.9800 - auc_7: 0.8409 - val_loss: 0.0770 - val_binary_accuracy: 0.9800 - val_auc_7: 0.8405
Epoch 3/10
83/83 [==============================] - 129s 2s/step - loss: 0.0760 - binary_accuracy: 0.9801 - auc_7: 0.8471 - val_loss: 0.0761 - val_binary_accuracy: 0.9800 - val_auc_7: 0.8464
Epoch 4/10
83/83 [==============================] - 130s 2s/step - loss: 0.0746 - binary_accuracy: 0.9802 - auc_7: 0.8577 - val_loss: 0.0753 - val_binary_accuracy: 0.9800 - val_auc_7: 0.8530
Epoch 5/10
83/83 [==============================] - 131s 2s/step - loss: 0.0732 - binary_accuracy: 0.9802 - auc_7: 0.8691 - val_loss: 0.0746 - val_binary_accuracy: 0.9800 - val_auc_7: 0.8594
Epoch 6/10
83/83 [===========================

In [ ]:
# Read in test data and preprocess similarly
os.chdir("/Users/zhongyuanchen/Downloads/cafa-5-protein-function-prediction/Test (Targets)")
test_fasta_sq = SeqIO.parse(open("testsuperset.fasta"),'fasta')
test_df = pd.DataFrame((fasta.id,str(fasta.seq)) for fasta in test_fasta_sq)
test_df.columns = ["EntryID","Sequence"]

# Trime each fasta sequence to have a TRIM_LENGTH size
test_df["Sequence"] = test_df["Sequence"].apply(lambda x:x[:TRIM_LENGTH] if len(x)>TRIM_LENGTH else x.ljust(TRIM_LENGTH,"0"))
test_df.set_index("EntryID",inplace=True)

# Similarly define generators for testing data
test_id_sq = {}
for entry_id,row in test_df.iterrows():
    test_id_sq[entry_id] = row["Sequence"]

# Define generator for testing
class Test_Data_Generator(tf.keras.utils.Sequence):

    def __init__(self,indexs, batch_size):
        self.indexs = indexs
        self.batch_size = batch_size

    def __len__(self):
        return math.floor(len(self.indexs) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.indexs))
        batch_x = []

        for i in range(low,high):
            entry_id = self.indexs[i]
            batch_x.append(sq_one_hot(test_id_sq[entry_id]))
            
        return np.array(batch_x).reshape(self.batch_size,TRIM_LENGTH,27,1)

test_indexs = list(test_df.index)
test_sq_generator = Test_Data_Generator(test_indexs,512)
prediction = model.predict(test_sq_generator)


In [ ]:
pred_df = []
for indx,entry_id in enumerate(test_indexs):
    for i in range(1500):
        pred_df.append([entry_id,terms_mlb.classes_[i],prediction[indx][i]])
    if indx%3000 == 0:
        print(indx)

submission = pd.DataFrame(pred_df)
submission.columns = ["EntryID","Terms","Prediction"]
submission.to_csv('submission.tsv', sep="\t",header = False,index = False)